<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [7]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [8]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code of the response:

In [9]:
print(response)

<Response [200]>


We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [10]:
  parameters={'lat': -33.87,'lon': 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [11]:
response = requests.get("http://api.open-notify.org/iss-pass.json", parameters)


Print the response header:

In [12]:
print(response)

<Response [200]>


Print the content of the response (the data that the server returned):

In [13]:
print(response.content)


b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1624600613, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 661, \n      "risetime": 1624600897\n    }, \n    {\n      "duration": 545, \n      "risetime": 1624606793\n    }, \n    {\n      "duration": 359, \n      "risetime": 1624612784\n    }, \n    {\n      "duration": 439, \n      "risetime": 1624618658\n    }, \n    {\n      "duration": 625, \n      "risetime": 1624624436\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [14]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [15]:
print(response.headers['content-type'])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [16]:
response.json()

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1624600613,
  'latitude': -33.87,
  'longitude': 151.21,
  'passes': 5},
 'response': [{'duration': 661, 'risetime': 1624600897},
  {'duration': 545, 'risetime': 1624606793},
  {'duration': 359, 'risetime': 1624612784},
  {'duration': 439, 'risetime': 1624618658},
  {'duration': 625, 'risetime': 1624624436}]}

What kind of object did this give us?

In [17]:
print(type(response.json()))

<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [39]:
iss_info = response.json()
print(iss_info)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1624600613, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 661, 'risetime': 1624600897}, {'duration': 545, 'risetime': 1624606793}, {'duration': 359, 'risetime': 1624612784}, {'duration': 439, 'risetime': 1624618658}, {'duration': 625, 'risetime': 1624624436}]}


Now extract the `risetime` strings into an array called `srisetimes`:

In [35]:
srisetimes = [row['risetime'] for row in iss_info['response']]
srisetimes 

[1624600897, 1624606793, 1624612784, 1624618658, 1624624436]

```
datetime.fromtimestamp(str)
```

Convert these to an array of Python `datetime` values called `risetimes`:

In [27]:
risetimes = list(map(datetime.fromtimestamp, srisetimes))

risetimes

[datetime.datetime(2021, 6, 25, 16, 1, 37),
 datetime.datetime(2021, 6, 25, 17, 39, 53),
 datetime.datetime(2021, 6, 25, 19, 19, 44),
 datetime.datetime(2021, 6, 25, 20, 57, 38),
 datetime.datetime(2021, 6, 25, 22, 33, 56)]

Finally, print these in a format that people understand:



```
str.strftime('%d/%m/%y %I:%M')

27/03/19 10:59
27/03/19 12:35
27/03/19 02:11
28/03/19 03:42
28/03/19 05:13
```



In [33]:
risetimes_f =[]

for i in risetimes:
    risetimes_f.append(i.strftime('%d/%m/%y %I:%M'))

risetimes_f
    


['25/06/21 04:01',
 '25/06/21 05:39',
 '25/06/21 07:19',
 '25/06/21 08:57',
 '25/06/21 10:33']

Here is an endpoint that tells us who is onboard:

In [43]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [44]:
print(response.content)

b'{"people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}, {"name": "Nie Haisheng", "craft": "Tiangong"}, {"name": "Liu Boming", "craft": "Tiangong"}, {"name": "Tang Hongbo", "craft": "Tiangong"}], "number": 10, "message": "success"}'


In [45]:
astr_info = response.json()
astr_info

{'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'},
  {'name': 'Oleg Novitskiy', 'craft': 'ISS'},
  {'name': 'Pyotr Dubrov', 'craft': 'ISS'},
  {'name': 'Thomas Pesquet', 'craft': 'ISS'},
  {'name': 'Megan McArthur', 'craft': 'ISS'},
  {'name': 'Shane Kimbrough', 'craft': 'ISS'},
  {'name': 'Akihiko Hoshide', 'craft': 'ISS'},
  {'name': 'Nie Haisheng', 'craft': 'Tiangong'},
  {'name': 'Liu Boming', 'craft': 'Tiangong'},
  {'name': 'Tang Hongbo', 'craft': 'Tiangong'}],
 'number': 10,
 'message': 'success'}

In [46]:
print(f"Number of astronauts is {astr_info['number']}")

Number of astronauts is 10


In [49]:
#Extract astronaut names

astr_names = [row['name'] for row in astr_info['people']]

astr_names 


['Mark Vande Hei',
 'Oleg Novitskiy',
 'Pyotr Dubrov',
 'Thomas Pesquet',
 'Megan McArthur',
 'Shane Kimbrough',
 'Akihiko Hoshide',
 'Nie Haisheng',
 'Liu Boming',
 'Tang Hongbo']

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [196]:
#ANSWER:
def handleResponse(response, verbose = False):
    
    print(response.status_code)
    return(response.status_code != 200, response.status_code) 
    
     
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [197]:
response = requests.get("http://api.open-notify.org/astrosq.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    

404
API call failed. Resolve issue before continuing!


In [166]:
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

True

>

>

>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



